In [ ]:
import jupyter_black

jupyter_black.load()
import pandas as pd
import nfl_data_py as nfl
from src.visualisation.visualisation import plot_bar, plot_scatter
from src.utils.methods import flatten_grouped_cols

pd.set_option("display.max_columns", None)

## Load Data

In [ ]:
pbp: pd.DataFrame = nfl.import_pbp_data([2023])

In [ ]:
# offense_players - list of id's
id_df = nfl.import_ids()
id_df.sample(4)

In [ ]:
wp_alpha = 0.05
pbp_pass = (
    pbp.query("week<=18")
    .query('play_type=="pass"')
    .query(f"wp >= {wp_alpha} and def_wp >={wp_alpha}")
)

# Target Shares

In [ ]:
[col for col in pbp_pass.columns if "yard" in col]

In [ ]:
target_df = (
    pbp_pass.groupby(["receiver_player_id", "receiver_player_name", "posteam"])
    .agg(
        {
            "play_id": "count",
            "complete_pass": "sum",
            "air_yards": "sum",
            "receiving_yards": "sum",
        }
    )
    .astype(int)
    .reset_index()
    .rename(
        columns={
            "play_id": "targets",
            "posteam": "team",
            "complete_pass": "receptions",
            "receiver_player_name": "player",
            "receiver_player_id": "player_id",
        }
    )
    .query("targets>=25")
    .sort_values("receptions", ascending=False)
)
target_df["team_targets"] = target_df.groupby("team")["targets"].transform("sum")
target_df["target_share"] = target_df["targets"] / target_df["team_targets"]
target_df = target_df.sort_values("target_share", ascending=False)
target_df.head(20)

In [ ]:
plot_bar(
    target_df.head(20),
    x="target_share",
    y="player",
    citation=False,
    title="Target Share 2023",
)